In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from caas_jupyter_tools import display_dataframe_to_user

ModuleNotFoundError: No module named 'caas_jupyter_tools'

In [ ]:
# Re-plot with X=Ingresos (parsed to numeric), Y=Grupo de Edad, Hue=POBREZA



df = pd.read_csv("base_proyecto.csv", sep = ",", index_col=0)
# If df isn't in memory (edge case), reload
try:
    df
except NameError:
    df = pd.read_csv("/mnt/data/base_proyecto.csv")

df0 = df.copy()

# --- Parse "Ingresos" to numeric robustly ---
def parse_ingresos(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip()
    # Remove currency symbols and spaces
    s1 = s.str.replace(r"[^\d,.\-]", "", regex=True)
    # Strategy A: assume comma is thousands separator -> remove commas
    a = pd.to_numeric(s1.str.replace(",", "", regex=False), errors="coerce")
    # Strategy B: assume comma is decimal separator -> swap , -> . and remove dots as thousands
    b = pd.to_numeric(
        s1.str.replace(".", "", regex=False).str.replace(",", ".", regex=False),
        errors="coerce"
    )
    # Choose the version with fewer NaNs (more successful parses)
    if a.notna().sum() >= b.notna().sum():
        return a
    else:
        return b

df0["Ingresos_num"] = parse_ingresos(df0["Ingresos"])

# Prepare data
required_cols = ["Grupo de Edad", "POBREZA", "Ingresos_num"]
plot_df = df0[required_cols].dropna()

# Cast hue to categorical for clearer legend labels
plot_df["POBREZA"] = plot_df["POBREZA"].astype(int)

# Plot
plt.figure(figsize=(9, 6))
for k, g in plot_df.groupby("POBREZA"):
    plt.scatter(g["Ingresos_num"], g["Grupo de Edad"], label=str(k), alpha=0.8, s=20)

plt.xlabel("Ingresos (num)")
plt.ylabel("Grupo de Edad")
plt.title("Ingresos vs Grupo de Edad por POBREZA")
plt.legend(title="POBREZA")
plt.tight_layout()
plt.show()

# Quick report on parsing success
report = pd.DataFrame({
    "rows_total":[len(df0)],
    "parsed_non_na":[plot_df.shape[0]],
    "dropped_rows":[len(df0) - plot_df.shape[0]]
})
display_dataframe_to_user("Parse report (Ingresos -> numeric)", report)